<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
    </a>
</p>


# Test Environment for Generative AI classroom labs

This lab provides a test environment for the codes generated using the Generative AI classroom.

Follow the instructions below to set up this environment for further use.


# Setup


### Import required libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

### Downloading the Dataset from UCI Archives
Use the code provided in the cell below to download the dataset. The dataset can also be found [here](https://archive.ics.uci.edu/dataset/45/heart+disease).

In [2]:
from ucimlrepo import fetch_ucirepo
import pandas as pd

uci_heart_disease_repo_id = 45

# fetch dataset 
heart_disease = fetch_ucirepo(id=uci_heart_disease_repo_id) 

# convert data to pandas dataframe
df = pd.concat([heart_disease.data.features, heart_disease.data.targets])

In [3]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,NaN
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,NaN
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,NaN
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,NaN
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,NaN


### Connect to the database

In [4]:
%load_ext sql

In [5]:
import sqlite3

con = sqlite3.connect("heart_disease.db")
cur = con.cursor()

In [6]:
%sql sqlite:///heart_disease.db

### write dataframe to the database

In [7]:
df.to_sql("heart_disease_prediction_dataset", con, if_exists='replace', index=False, method="multi")

606

---


# Context

We have a Heart Disease prediction dataset with a single table which has the following attributes.
1. age - age in years
2. sex - sex (1 = male; 0 = female)
3. cp - chest pain type
        -- Value 1: typical angina
        -- Value 2: atypical angina
        -- Value 3: non-anginal pain
        -- Value 4: asymptomatic
4. trestbps - resting blood pressure (in mm Hg on admission to the hospital)
5. chol - serum cholestoral in mg/dl
6. fbs - (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
7. restecg - resting electrocardiographic results
        -- Value 0: normal
        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
        -- Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
8. thalach - maximum heart rate achieved
9. exang - exercise induced angina (1 = yes; 0 = no)
10. oldpeak - ST depression induced by exercise relative to rest
11. slope - the slope of the peak exercise ST segment
        -- Value 1: upsloping
        -- Value 2: flat
        -- Value 3: downsloping
12. ca - number of major vessels (0-3) colored by flourosopy
13. thal - 3 = normal; 6 = fixed defect; 7 = reversable defect
14. num (the predicted attribute) - diagnosis of heart disease (angiographic disease status)
        -- Value 0: < 50% diameter narrowing
        -- Value 1: > 50% diameter narrowing

# Prompts for Data Querying

## Age Distribution

### Prompt

Write an SQL query to find the minimum, maximum, and average age of patients in the dataset.

### Generated Code

In [8]:
%%sql
SELECT 
    MIN(age) AS min_age,
    MAX(age) AS max_age,
    AVG(age) AS avg_age
FROM 
    heart_disease_prediction_dataset;

 * sqlite:///heart_disease.db
Done.


min_age,max_age,avg_age
29.0,77.0,54.43894389438944


## Gender Analysis

### Prompt

Write and SQL query to count the number of male and female patients in the dataset.

### Generated Code

In [9]:
%%sql
SELECT 
    sex,
    COUNT(*) AS patient_count
FROM 
    heart_disease_prediction_dataset
GROUP BY 
    sex;

 * sqlite:///heart_disease.db
Done.


sex,patient_count
None,303
0.0,97
1.0,206


## Chest Pain Type Frequency

### Prompt

Write an SQL query to determine the frequency of each type of chest pain (typical angina, atypical angina, non-anginal pain, asymptomatic) among patients."

### Generated Code

In [10]:
%%sql
SELECT 
    cp,
    COUNT(*) AS pain_frequency
FROM 
    heart_disease_prediction_dataset
GROUP BY 
    cp;

 * sqlite:///heart_disease.db
Done.


cp,pain_frequency
None,303
1.0,23
2.0,50
3.0,86
4.0,144


## Age Group Analysis and Target Variable

### Prompt

Write an SQL query to investigate the distribution of the target variable (presence or absence of heart disease) within different age groups (e.g., 20-30, 30-40, etc.).

### Generated Code

In [11]:
%%sql
SELECT 
    CASE
        WHEN age BETWEEN 20 AND 30 THEN '20-30'
        WHEN age BETWEEN 31 AND 40 THEN '31-40'
        WHEN age BETWEEN 41 AND 50 THEN '41-50'
        WHEN age BETWEEN 51 AND 60 THEN '51-60'
        WHEN age BETWEEN 61 AND 70 THEN '61-70'
        ELSE 'Above 70'
    END AS age_group,
    SUM(CASE WHEN num = 1 THEN 1 ELSE 0 END) AS heart_disease_count,
    SUM(CASE WHEN num = 0 THEN 1 ELSE 0 END) AS no_heart_disease_count
FROM 
    heart_disease_prediction_dataset
GROUP BY 
    age_group
ORDER BY 
    age_group;

 * sqlite:///heart_disease.db
Done.


age_group,heart_disease_count,no_heart_disease_count
20-30,0,0
31-40,0,0
41-50,0,0
51-60,0,0
61-70,0,0
Above 70,55,164


# Practice Prompts

## Cholesterol Range

### Prompt

Find the range of cholesterol levels among patients (minimum, maximum).

### Generated Code

In [12]:
%%sql
SELECT MIN(chol) AS Minimum_Cholesterol_Level,
       MAX(chol) AS Maximum_Cholesterol_Level
FROM heart_disease_prediction_dataset;

 * sqlite:///heart_disease.db
Done.


Minimum_Cholesterol_Level,Maximum_Cholesterol_Level
126.0,564.0


## Age Range and Gender Analysis

### Prompt

Determine the age range (youngest and oldest) for male and female patients separately.

### Generated Code

In [13]:
%%sql
SELECT MIN(age) AS Youngest_Age,
       MAX(age) AS Oldest_Age
FROM heart_disease_prediction_dataset
WHERE sex = 1;

 * sqlite:///heart_disease.db
Done.


Youngest_Age,Oldest_Age
29.0,77.0


In [14]:
%%sql
SELECT MIN(age) AS Youngest_Age,
       MAX(age) AS Oldest_Age
FROM heart_disease_prediction_dataset
WHERE sex = 0;

 * sqlite:///heart_disease.db
Done.


Youngest_Age,Oldest_Age
34.0,76.0


## Age Group Analysis and Target Variable

### Prompt

Investigate the distribution of the target variable (presence or absence of heart disease) within different age groups (e.g., 20-30, 30-40, etc.).

### Generated Code

In [15]:
%%sql
SELECT 
    CASE 
        WHEN age >= 20 AND age < 30 THEN '20-29'
        WHEN age >= 30 AND age < 40 THEN '30-39'
        WHEN age >= 40 AND age < 50 THEN '40-49'
        WHEN age >= 50 AND age < 60 THEN '50-59'
        WHEN age >= 60 AND age < 70 THEN '60-69'
        ELSE '70 and above'
    END AS Age_Group,
    SUM(CASE WHEN num = 1 THEN 1 ELSE 0 END) AS Presence_of_Heart_Disease,
    SUM(CASE WHEN num = 0 THEN 1 ELSE 0 END) AS Absence_of_Heart_Disease
FROM heart_disease_prediction_dataset
GROUP BY Age_Group
ORDER BY Age_Group;

 * sqlite:///heart_disease.db
Done.


Age_Group,Presence_of_Heart_Disease,Absence_of_Heart_Disease
20-29,0,0
30-39,0,0
40-49,0,0
50-59,0,0
60-69,0,0
70 and above,55,164


## Maximum Heart Rate by Age Group

### Prompt

Find the maximum heart rate achieved during exercise for different age groups (e.g., 30-40, 40-50, etc.).

### Generated Code

In [16]:
%%sql
SELECT 
    CASE 
        WHEN age >= 30 AND age < 40 THEN '30-39'
        WHEN age >= 40 AND age < 50 THEN '40-49'
        WHEN age >= 50 AND age < 60 THEN '50-59'
        WHEN age >= 60 AND age < 70 THEN '60-69'
        ELSE '70 and above'
    END AS Age_Group,
    MAX(thalach) AS Max_Heart_Rate
FROM heart_disease_prediction_dataset
GROUP BY Age_Group
ORDER BY Age_Group;

 * sqlite:///heart_disease.db
Done.


Age_Group,Max_Heart_Rate
30-39,192.0
40-49,194.0
50-59,195.0
60-69,179.0
70 and above,202.0


## Percentage of Patients with High Blood Sugar

### Prompt

Calculate the percentage of patients with fasting blood sugar greater than 120 mg/dl.

### Generated Code

In [17]:
%%sql
SELECT 
    (SUM(CASE WHEN fbs = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS Percentage_FBS_Greater_Than_120
FROM heart_disease_prediction_dataset;

 * sqlite:///heart_disease.db
Done.


Percentage_FBS_Greater_Than_120
7.425742574257426


## Ratio of Patients with Resting Electrocardiographic Abnormality

### Prompt

Find the ratio of patients with abnormal resting electrocardiographic results to those with normal results.

### Generated Code

In [18]:
%%sql
SELECT 
    (SUM(CASE WHEN restecg > 0 THEN 1 ELSE 0 END) * 1.0) / 
    (SUM(CASE WHEN restecg = 0 THEN 1 ELSE 0 END) * 1.0) AS Ratio_Abnormal_to_Normal
FROM heart_disease_prediction_dataset;

 * sqlite:///heart_disease.db
Done.


Ratio_Abnormal_to_Normal
1.0066225165562914


## Number of Patients with Reversible Thalassemia

### Prompt

Count the number of patients with reversible thalassemia detected by thallium stress testing.

### Generated Code

In [19]:
%%sql
SELECT COUNT(*) AS Count_Reversible_Thalassemia
FROM heart_disease_prediction_dataset
WHERE thal = 7;

 * sqlite:///heart_disease.db
Done.


Count_Reversible_Thalassemia
117


## Average Age of Patients with Chest Pain

### Prompt

Calculate the average age of patients who experienced chest pain during diagnosis.

### Generated Code

In [20]:
%%sql
SELECT AVG(age) AS Average_Age_With_Chest_Pain
FROM heart_disease_prediction_dataset
WHERE cp IN (1, 2, 3);

 * sqlite:///heart_disease.db
Done.


Average_Age_With_Chest_Pain
53.276729559748425


## Distribution of Patients by Number of Major Vessels

### Prompt

Investigate the distribution of patients based on the number of major vessels colored by fluoroscopy (0-3).

### Generated Code

In [21]:
%%sql
SELECT ca AS Number_of_Vessels, COUNT(*) AS Patient_Count
FROM heart_disease_prediction_dataset
GROUP BY ca
ORDER BY ca;

 * sqlite:///heart_disease.db
Done.


Number_of_Vessels,Patient_Count
None,307
0.0,176
1.0,65
2.0,38
3.0,20


## Authors


[Abhishek Gagneja](https://www.linkedin.com/in/abhishek-gagneja-23051987/)


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-12-10|0.1|Abhishek Gagneja|Initial Draft created|


Copyright © 2023 IBM Corporation. All rights reserved.
